In [2]:
# producer.py

from confluent_kafka import Producer
import pandas as pd
import time
import json

file_path = 'air_quality_index.csv'
df = pd.read_csv(file_path)

bootstrap_servers = 'localhost:9092'
topic = 'air-quality-test1'

producer_config = {
    'bootstrap.servers': bootstrap_servers,
    'client.id': 'kafka-producer.csv'
}

producer = Producer(producer_config)

def send_data_to_kafka(row):
    message = json.dumps({"id": row['Unnamed: 0'], "data": row.to_dict()})
    producer.produce(topic, key=str(row['Unnamed: 0']), value=message)
    producer.flush()
    time.sleep(2)  # Adjust delay based on your requirements

for index, row in df.iterrows():
    send_data_to_kafka(row)

producer.close()


KeyboardInterrupt: 

In [1]:
# spark_streaming_mongodb_consumer.py

from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
#from pyspark.streaming.kafka import KafkaUtils
import json
#from pymongo import MongoClient
import pymongo
import os 
import os

os.environ['HADOOP_HOME']='C:\HADOOP'
os.environ['PATH']='%PATH%;%HADOOP_HOME%/bin;'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'


def process_stream(rdd):
    # Check if the RDD is not empty
    if not rdd.isEmpty():
        # Print the data in the Jupyter console
        rdd.foreach(lambda record: print(json.loads(record[1])))

        # Create a MongoClient to connect to the MongoDB server
        mongo_client = MongoClient("mongodb://localhost:27017/")

        # Connect to the 'air_quality_db' database and 'air_quality_collection' collection
        db = mongo_client["air_quality_db"]
        collection = db["air_quality_collection"]

        # Convert each record in the RDD to a JSON object and insert into MongoDB
        rdd.foreach(lambda record: collection.insert_one(json.loads(record[1])))

        # Close the MongoClient connection
        mongo_client.close()



        
scala_version = '2.12'
spark_version = '3.1.2'
# TODO: Ensure match above values match the correct versions
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.2.1',
    'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1'
]        
        
        
             
        
        
        
        
# Initialize SparkSession
spark = SparkSession.builder.appName("AirQualityStreaming").config("spark.jars.packages",",".join(packages)).getOrCreate()

# Create StreamingContext with a batch interval of 2 seconds
ssc = StreamingContext(spark.sparkContext, 2)

# Kafka parameters
kafka_params = {
    "bootstrap.servers": "localhost:9092",
    "group.id": "spark-streaming-group",
    "auto.offset.reset": "earliest"
}

# Kafka topic
topic = "air-quality-test1"
'''
# Create a DStream from Kafka
kafka_stream = KafkaUtils.createDirectStream(ssc, [topic], kafka_params)

'''
kafka_stream=spark.readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers",'localhost:9092') \
                .option("subscribe","air-quality-test1") \
                .load()



# Process each RDD and print to console and save to MongoDB using foreachRDD
#kafka_stream.foreachRDD(process_stream)



#'''
writeStream = kafka_stream \
        .writeStream \
        .outputMode("append") \
        .foreachBatch(process_stream) \
        .option("checkpointLocation", "/tmp/spark-checkpoint") \
        .start()

writeStream.awaitTermination()

#'''


# Start the streaming context
#ssc.start()
#ssc.awaitTermination()


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.